In this notebook we will instead look at saving data to csv files instead of a local database. This would allow us to access these from a deployed web version of our scripts with live prediction displays and visual up-to-date model performance metrics.

In [2]:
import pandas as pd
import sqlite3
from sqlite3 import Error


def from_acc_db():
    '''
    This function recalls our model metrics database and plots various values and metrics for model performance over time.
    '''
    
    conn = None
    try:
        conn = sqlite3.connect('bitcoin_model.db')
        print('Connected Successfully!')
    
    except Error as e:
        print(e)
        
    query = """
            SELECT *
            FROM performance
            """
    
    data = pd.read_sql(query, conn, parse_dates=['Date'])        # adding in the query, connection, and parsing date column as its correct format
    data.set_index('Date', inplace=True)                         # setting the index of the resulting dataframe to the date column
    conn.close()                                                 # closiung connection to database
    
    return data

In [2]:
data = from_acc_db()
data

Connected Successfully!


,Close,Prediction,Confidence,True_Label,Correct_Pred,Model_Accuracy
Date,,,,,,
2021-12-05,49396.33,1,0.55,1,1,100
2021-12-06,50441.92,1,0.71,1,1,100
2021-12-07,50588.95,1,0.60,1,1,100
2021-12-08,50471.19,1,0.77,0,0,75
2021-12-09,47545.59,1,0.85,1,1,80


In [3]:
data.to_csv('first_csv.csv', index_label='Date')

In [8]:
!pip3 install google-api-python-client

     |████████████████████████████████| 7.9 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 132.5 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 35.2 MB/s  eta 0:00:01
     |████████████████████████████████| 207 kB 117.1 MB/s eta 0:00:01


In [12]:
!pip3 install google-cloud-storage

     |████████████████████████████████| 106 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 9.8 MB/s  eta 0:00:01


In [19]:
from google.cloud import storage

client = storage.Client()
bucket = client.bucket('my_perfect_model_data')
blob = bucket.blob('csvs')
blob.upload_from_filename(filename='first_csv.csv')

# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/[bucket-id]/
# bucket = client.get_bucket('bucket-id')
# # Then do other things...
# blob = bucket.get_blob('remote/path/to/file.txt')
# print(blob.download_as_bytes())
# blob.upload_from_string('New contents!')


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [4]:
# this did not work very well. was having issues setting permisssions. Will have to come back to this section, for now, 
# will focus on accessing csv stored directly in google cloud so that our deployable app will work from anywhere.


test_df = pd.read_csv('https://storage.googleapis.com/my_perfect_model_data/first_csv.csv',  index_col='Date', parse_dates=True)
test_df

,Close,Prediction,Confidence,True_Label,Correct_Pred,Model_Accuracy
Date,,,,,,
2021-12-05,49396.33,1,0.55,1,1,100
2021-12-06,50441.92,1,0.71,1,1,100
2021-12-07,50588.95,1,0.60,1,1,100
2021-12-08,50471.19,1,0.77,0,0,75
2021-12-09,47545.59,1,0.85,1,1,80


In [5]:
test_df.dtypes

Close             float64
Prediction          int64
Confidence        float64
True_Label          int64
Correct_Pred        int64
Model_Accuracy      int64
dtype: object

In [ ]:
# now that we know we can properly access our stored csv files, we can rework our previous functions
# we can then also integrate these functions into our streamlit application process to display our desired outputs


